# CubeDynamics Quickstart: Pipe-based Climate Cube Operations


Install CubeDynamics from GitHub (run in your terminal or a notebook cell before importing):

```bash
pip install "git+https://github.com/CU-ESIIL/climate_cube_math.git@main"
```

If you prefer running inside the notebook, uncomment the cell below.


In [ ]:
# Optional notebook install
# !pip install "git+https://github.com/CU-ESIIL/climate_cube_math.git@main"


## Canonical minimal example

Copy/paste this entire cell into a blank notebook to create a cube, run anomaly/month filtering, and compute variance.


In [ ]:
import numpy as np
import pandas as pd
import xarray as xr
import cubedynamics as cd

# 1. Create a tiny example "cube" with a datetime coordinate
time = pd.date_range("2000-01-01", periods=12, freq="MS")
values = np.arange(12, dtype=float)

cube = xr.DataArray(
    values,
    dims=["time"],
    coords={"time": time},
    name="example_variable",
)

# 2. Run a pipe chain: anomaly -> month filter -> variance
result = (
    cd.pipe(cube)
    | cd.anomaly(dim="time")
    | cd.month_filter([6, 7, 8])
    | cd.variance(dim="time")
).unwrap()

print("Variance of anomalies over JJA:", float(result.values))


## Filter months and write to NetCDF

Each example below is self-contained. This one recreates imports/data, filters summer months, and writes them to a temporary NetCDF file.


In [ ]:
import numpy as np
import pandas as pd
import xarray as xr
import cubedynamics as cd
from pathlib import Path
import tempfile

# Monthly datetime coordinate ensures month_filter works
monthly_time = pd.date_range("2001-01-01", periods=12, freq="MS")
monthly_values = np.arange(monthly_time.size, dtype=float)

monthly_cube = xr.DataArray(
    monthly_values,
    dims=["time"],
    coords={"time": monthly_time},
    name="monthly_example",
)

with tempfile.TemporaryDirectory() as tmpdir:
    out_path = Path(tmpdir) / "summer.nc"
    summer = (
        cd.pipe(monthly_cube)
        | cd.month_filter([6, 7, 8])
        | cd.to_netcdf(out_path)
    ).unwrap()

    print("Saved summer subset to:", out_path)
    print(summer)
